# IMSNG_Rougine.ipynb
- Automatically Process the image data from GECKO facilities and search for transients in the subtracted images with `gpPy`
- Author: Gregory S.H. Paek (23.04.24)
- Update: Gregory S.H. Paek (23.05.23)

## Library

In [1]:
from __future__ import print_function, division, absolute_import
import os, sys, glob, subprocess
import numpy as np
import astropy.io.ascii as ascii
import matplotlib.pyplot as plt
plt.ioff()
from astropy.nddata import CCDData
from preprocess import calib
from util import tool
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table, vstack
from astropy import units as u
from ccdproc import ImageFileCollection
import warnings
warnings.filterwarnings(action='ignore')
# from itertools import product
from itertools import repeat
import multiprocessing
import time
start_localtime = time.strftime('%Y-%m-%d %H:%M:%S (%Z)', time.localtime())

In [2]:
#	plot setting
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["axes.labelsize"] = 20
plt.rcParams['savefig.dpi'] = 500
plt.rc('font', family='serif')

## Ready

In [3]:
# try:
# 	obs = (sys.argv[1]).upper()
# except:
# 	obs = input(f"OBS TO PROCESS:").upper()

# obs = 'KCT_STX16803'
obs = 'LSGT_ASI1600MM'
print(f'# Observatory : {obs.upper()}')

try:
	ncores = int(sys.argv[2])
except:
	ncores = 1
print(f"- Number of Cores: {ncores}")

# Observatory : LSGT_ASI1600MM
- Number of Cores: 1


### Path

In [4]:
path_base = '/data4/gecko/factory'
path_ref = f'{path_base}/ref_frames/{obs.upper()}'
path_factory = f'{path_base}/{obs.lower()}'
path_save = f'/data6/bkgdata/{obs.upper()}'
path_log = f'/home/paek/log/{obs.lower()}.log'
path_keys = '/home/paek/table'
#------------------------------------------------------------
path_gal = '/data6/IMSNG/IMSNGgalaxies'
path_refcat = '/data4/gecko/factory/ref_frames/LOAO'
#------------------------------------------------------------
# path_config = '/home/paek/config'
path_config = './config'
path_default_gphot = f'{path_config}/gphot.{obs.lower()}.config'
# path_mframe = f'{path_base}/master_frames'
path_mframe = f'/data3/paek/factory/master_frames'
path_calib = f'{path_base}/calib'
#------------------------------------------------------------
#	Codes
#------------------------------------------------------------
path_phot_sg = './phot/gregoryphot_2021.py'
path_phot_mp = './phot/gregoryphot_mp_2021.py'
path_phot_sub = './phot/gregoryphot_sub_2021.py'
path_find = './phot/gregoryfind_bulk_mp_2021.py'
#------------------------------------------------------------
if 'KCT_ASI1600MM' in obs:
	path_raw = '/data3/IMSNG/KCT/obsdata'
else:
	path_raw = f'/data6/obsdata/{obs.upper()}'
rawlist = sorted(glob.glob(path_raw+'/2*'))
#------------------------------------------------------------
path_obs = '/home/paek/table/obs.dat'
path_changehdr = '/home/paek/table/changehdr.dat'
path_alltarget = '/home/paek/table/alltarget.dat'
ccdinfo = tool.getccdinfo(obs, path_obs)

- Folder setting

In [5]:
if not os.path.exists(path_base):
	os.makedirs(path_base)
if not os.path.exists(path_ref):
	os.makedirs(path_ref)
if not os.path.exists(path_factory):
	os.makedirs(path_factory)
if not os.path.exists(path_save):
	os.makedirs(path_save)

### Table

In [6]:
logtbl = ascii.read(path_log)
datalist = np.copy(logtbl['date'])
obstbl = ascii.read(path_obs)
hdrtbl = ascii.read(path_changehdr)
alltbl = ascii.read(path_alltarget)
keytbl = ascii.read(f'{path_keys}/keys.dat')

## Process Summary Status

In [7]:
protbl = Table()
protbl['process'] = ['master_frame', 'pre_process', 'astrometry', 'cr_removal', 'defringe', 'photometry', 'image_stack', 'photometry_com', 'subtraction', 'photometry_sub', 'transient_search', 'total']
protbl['status'] = False
protbl['time'] = 0.0 * u.second

## Main Body

In [8]:
newlist = [i for i in rawlist if (i not in datalist) & (i+'/' not in datalist)]
if len(newlist) == 0:
	print('No new data')
	sys.exit()
else:
	print(newlist)

# path = newlist[1]
#	Test folder
path = "/data6/obsdata/LSGT_ASI1600MM/gwtest2"
tdict = dict()
starttime = time.time()
path_data = '{}/{}'.format(path_factory, os.path.basename(path))
#	Remove old folder and re-copy folder
rmcom = 'rm -rf {}'.format(path_data)
print(rmcom)
os.system(rmcom)
cpcom = 'cp -r {} {}'.format(path, path_data)
print(cpcom)
os.system(cpcom)

['/data6/obsdata/LSGT_ASI1600MM/20230520', '/data6/obsdata/LSGT_ASI1600MM/20230521', '/data6/obsdata/LSGT_ASI1600MM/20230523', '/data6/obsdata/LSGT_ASI1600MM/20230524', '/data6/obsdata/LSGT_ASI1600MM/20230526', '/data6/obsdata/LSGT_ASI1600MM/20230528', '/data6/obsdata/LSGT_ASI1600MM/20230529', '/data6/obsdata/LSGT_ASI1600MM/20230530', '/data6/obsdata/LSGT_ASI1600MM/20230531']
rm -rf /data4/gecko/factory/lsgt_asi1600mm/gwtest2
cp -r /data6/obsdata/LSGT_ASI1600MM/gwtest2 /data4/gecko/factory/lsgt_asi1600mm/gwtest2


0

### Header Correction

In [9]:
ic0 = ImageFileCollection(path_data, keywords='*')
ic0.summary.write('{}/hdr.raw.dat'.format(path_data), format='ascii.tab', overwrite=True) 
#	Exceptions
#	DOAO CCD
if obs == 'DOAO':
	instrume = ic0.summary['instrume'][0]
	if instrume == 'Apogee USB/Net':obs = 'DOAO_APOGEE'
	elif instrume == '':			obs = 'DOAO_FLI'
	elif instrume == 'FLI':			obs = 'DOAO_FLI'
	elif instrume == 'Moravian C4-16000':		obs = 'DOAO_C416K'
else:
	pass
obsinfo = calib.getobsinfo(obs, obstbl)
if obs == 'SAO_C361K':
	xbinning = ic0.summary['xbinning'][0]
	if xbinning > 1:
		print(f'{obs} : BINNINGx{xbinning}')
		obsinfo['pixscale'] = obsinfo['pixscale']*xbinning
	if ic0.summary['instrume'][0] == 'ASCOM Camera Driver for FLI Kepler':
		obsinfo['pixscale'] = 0.311
		obsinfo['fov'] = 0.25*60.
		obsinfo['gain'] = float(ic0.summary['egain'][0])
#	KHAO Binning
if obs == 'KHAO':
	xbinning = ic0.summary['xbinning'][0]
	if xbinning > 1:
		obsinfo['pixscale'] = obsinfo['pixscale']*2
#	LOAO
if obs == 'LOAO':
	instrume = ic0.summary['instrume'][0]
	if 'Finger Lakes' in instrume:
		obs = 'LOAO_FLI'
		obsinfo = calib.getobsinfo(obs, obstbl)
#	RASA Mode
if obs == 'RASA36':
	if 'hdr' in path:
		mode = 'hdr'
		badmode = True
	elif 'high' in path:
		mode = 'high'
		badmode = True
	else:
		mode = 'high'
		badmode = True
		pass
	print(f'Master Frame Mode:{mode} [Bad Mode:{badmode}]')

calib.correcthdr_routine(path_data, hdrtbl, obs)
print("Correction Done")
# objfilterlist, objexptimelist, flatfilterlist, darkexptimelist, obstime = calib.correcthdr_routine(path_data, hdrtbl)
ic1 = ImageFileCollection(path_data, keywords='*')
ic1.summary.write('{}/hdr.cor.dat'.format(path_data), format='ascii.tab', overwrite=True)
try:
	nobj = len(ic1.filter(imagetyp='OBJECT').summary)
except:
	nobj = len(ic1.filter(imagetyp='object').summary)	

------------------------------------------------------------
CHANGE TO UNIVERSAL HEADER SERIES ...
------------------------------------------------------------

/data4/gecko/factory/lsgt_asi1600mm/gwtest2/T52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/T52-ceouobs.sophia-S230518h-20230519-001052-r-BIN1-E-180-002.fit
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/T52-ceouobs.sophia-S230518h-20230519-001406-r-BIN1-E-180-003.fit
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/T52-ceouobs.sophia-S230518h-20230519-011216-r-BIN1-E-180-001.fit
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/T52-ceouobs.sophia-S230518h-20230519-011530-r-BIN1-E-180-002.fit
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/T52-ceouobs.sophia-S230518h-20230519-011844-r-BIN1-E-180-003.fit
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/T52-ceouobs.sophia-S230518h-20230519-020455-r-BIN1-E-180-001.fit
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/T52-ceouobs.sophia-S230518h-2023

### Marking the `GECKO` data

In [10]:
gwobjlist = []

# testobj = 'S190425z'
project = "IMSNG"
obsmode = "MONITORING" # Default
for obj in ic1.filter(imagetyp='object').summary['object']:
	if 'MS' in obj[:2]: # MS230425 (test event)
		print(obj)
		project = "GECKO"
		gwobjlist.append(obj)
	elif 'S2' in obj[:2]: # S230425 (super event)
		print(obj)
		project = "GECKO"
		gwobjlist.append(obj)
	else:
		pass

#	OBSERVATION MODE
if obs in ['KCT_STX16803', 'RASA36', 'CBNUO',]:
	obsmode = "TILING"
else:
	obsmode = "GALTARGET"

print(f"[{project}] {obsmode}")
if obsmode == 'TILING':
	if obs == 'KCT_STX16803':
		path_skygrid = f'./data/skygrid/KCT/displaycenter.txt'
	else:
		path_skygrid = f'./data/skygrid/{obs}/displaycenter.txt'
	skygridtbl = Table.read(path_skygrid, format='csv')
	c_skygrid = SkyCoord(skygridtbl['ra'], skygridtbl['dec'], unit='deg')
	print(f"GET SKYGRID: {path_skygrid}")
else:
	pass

if project == 'GECKO':
	if len(set(gwobjlist)) == 1:
		gwobj = gwobjlist[0]
		print(f"[{project}] Correct GW event: {gwobj}")
	else:
		gwobj = gwobjlist[0]
		print(f"[{project}] There are {len(set(gwobjlist))} type of data --> Use {gwobj}...")

S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
S230518h
[GECKO] GALTARGET
[GECKO] Correct GW event: S230518h


In [11]:
path_LVK_Alert = f"/data7/GECKO/LVK_alert"
alert_type_list = ['PRELIMINARY', 'INITIAL', 'UPDATE']
alert_type = 'PRELIMINARY'

for _alert_type in alert_type_list:
	#	GW event path from `GeckoDigestor`
	_path_gwevent = f'{path_LVK_Alert}/{gwobj}_{_alert_type}'
	#	Host galaxy catalog
	_path_hgcat = f"{_path_gwevent}/HostGalaxyCatalog_90.csv"
	if os.path.exists(_path_gwevent) & os.path.exists(_path_hgcat):
		#	Update the GW info
		path_gwevent = _path_gwevent
		alert_type = _alert_type
		path_hgcat = _path_hgcat

print(f"Connect to GW Host Galaxy Catalog --> {path_hgcat}")
hgtbl = Table.read(path_hgcat, format='csv')
c_hg = SkyCoord(hgtbl['ra'], hgtbl['dec'], unit='deg')

path_ps1 = '/data4/ps1gal'
ps1tbl = Table.read(f'{path_ps1}/complete_download_total.fits')

Connect to GW Host Galaxy Catalog --> /data7/GECKO/LVK_alert/S230518h_UPDATE/HostGalaxyCatalog_90.csv


- Slack notification

In [12]:
OAuth_Token = keytbl['key'][keytbl['name']=='slack'].item()

channel = '#pipeline'
text = f'[`gpPy`/{project}-{obsmode}] Start Processing {obs} {os.path.basename(path)} Data ({nobj} objects) with {ncores} cores'

param_slack = dict(
	token = OAuth_Token,
	channel = channel,
	text = text,
)

tool.slack_bot(**param_slack)

<Response [200]>


### Master Frame

- Bias

In [13]:
st = time.time()
#------------------------------------------------------------
#	BIAS
#------------------------------------------------------------
try:
	biasnumb = len(ic1.filter(imagetyp='Bias').summary)
except:
	biasnumb = 0
# if len(ic1.filter(imagetyp='Bias').summary) != 0:
if biasnumb != 0:
	mzero = calib.master_zero(ic1, fig=False)
	# print(zeroim)
	date = fits.getheader(f'{path_data}/zero.fits')['date-obs'][:10].replace('-', '')
	if obs == 'RASA36':
		zeroim = f'{path_mframe}/{obs}/zero/{date}-zero_{mode}.fits'
	else:
		zeroim = f'{path_mframe}/{obs}/zero/{date}-zero.fits'
	cpcom = f'cp {path_data}/zero.fits {zeroim}'
	print(cpcom)
	os.system(cpcom)
	plt.close('all')
else:
	#	IF THERE IS NO FLAT FRAMES, BORROW FROM CLOSEST OTHER DATE
	print('\nNO BIAS FRAMES\n')
	if obs == 'RASA36':
		pastzero = np.array(glob.glob(f'{path_mframe}/{obs}/zero/*zero_{mode}.fits'))
	else:
		pastzero = np.array(glob.glob(f'{path_mframe}/{obs}/zero/*zero.fits'))
	#	CALCULATE CLOSEST ONE FROM TIME DIFFERENCE
	deltime = []
	for date in pastzero:
		zeromjd = calib.isot_to_mjd((os.path.basename(date)).split('-')[0])
		deltime.append(np.abs(ic1.summary['mjd'][0]-zeromjd))
	indx_closet = np.where(deltime == np.min(deltime))
	# tmpzero = path_data+'/'+os.path.basename(np.asscalar(pastzero[indx_closet]))
	tmpzero = f"{path_data}/{os.path.basename(pastzero[indx_closet][0])}"
	# cpcom = 'cp {} {}'.format(np.asscalar(pastzero[indx_closet]), tmpzero)
	cpcom = f'cp {pastzero[indx_closet][0]} {tmpzero}'
	print(cpcom)
	os.system(cpcom)
	# if obs != 'KCT':
	if 'KCT_ASI1600MM' in obs:
		#KCT Exception
		mzero = CCDData.read(tmpzero, hdu=0, unit='adu')
	elif obs == 'RASA36':
		if (mode == 'high') & (badmode == True):
			mzero = CCDData.read(tmpzero, hdu=0).multiply(20)
			print('[Bad mode] Multiply 20 on the high mode bias.')
		else:
			mzero = CCDData.read(tmpzero, hdu=0)
	elif obs == 'LSGT':
		mzero = CCDData.read(tmpzero, hdu=0, unit='adu')
	else:
		mzero = CCDData.read(tmpzero, hdu=0)#, unit='adu')
	mzero.meta['FILENAME'] = os.path.basename(tmpzero)


NO BIAS FRAMES

cp /data3/paek/factory/master_frames/LSGT_ASI1600MM/zero/20230515-zero.fits /data4/gecko/factory/lsgt_asi1600mm/gwtest2/20230515-zero.fits


- Dark

In [14]:
#------------------------------------------------------------
#	DARK (ITERATION FOR EACH EXPOSURE TIMES)
#------------------------------------------------------------
try:
	darkexptimelist = sorted(list(set(ic1.filter(imagetyp='dark').summary['exptime'])))
	darknumb = len(darkexptimelist)
except:
	darknumb = 0
darkdict = dict()
# if len(darkexptimelist) != 0:
if darknumb != 0:
	dark_process = True
	for i, exptime in enumerate(darkexptimelist):
		print('PRE PROCESS FOR DARK ({} sec)\t[{}/{}]'.format(exptime, i+1, len(darkexptimelist)))
		mdark = calib.master_dark(ic1, mzero=mzero, exptime=exptime, fig=False)
		darkdict['{}'.format(int(exptime))] = mdark

		date = fits.getheader(f'{path_data}/dark-{int(exptime)}.fits')['date-obs'][:10].replace('-', '')
		if obs == 'RASA36':
			darkim = f'{path_mframe}/{obs}/dark/{int(exptime)}-{date}-dark_{mode}.fits'
		else:
			darkim = f'{path_mframe}/{obs}/dark/{int(exptime)}-{date}-dark.fits'
		# print(zeroim)
		cpcom = 'cp {}/dark-{}.fits {}'.format(path_data, int(exptime), darkim)
		print(cpcom)
		os.system(cpcom)
		plt.close('all')
else:
	#	Borrow
	print('\nNO DARK FRAMES\n')
	objexptimelist = sorted(list(set(ic1.filter(imagetyp='object').summary['exptime'])))
	exptime = objexptimelist[-1]
	# pastdark = np.array(glob.glob('{}/{}/dark/{}*dark*.fits'.format(path_mframe, obs, int(exptime))))
	if obs == 'RASA36':
		pastdark = np.array(glob.glob(f'{path_mframe}/{obs}/dark/{int(exptime)}*dark_{mode}.fits'))
	else:
		pastdark = np.array(glob.glob(f'{path_mframe}/{obs}/dark/{int(exptime)}*dark.fits'))

	if len(pastdark) == 0:
		pastdark = np.array(glob.glob('{}/{}/dark/*dark*.fits'.format(path_mframe, obs)))
	else:
		pass
	#	CALCULATE CLOSEST ONE FROM TIME DIFFERENCE
	deltime = []
	delexptime = []
	darkexptimes = []
	for date in pastdark:
		# darkmjd = calib.isot_to_mjd((os.path.basename(date)).split('-')[0])
		darkmjd = calib.isot_to_mjd((os.path.basename(date)).split('-')[1])
		darkexptime = int( os.path.basename(date).split('-')[0] )
		# darkexptime = delexptime.append(int( os.path.basename(date).split('-')[1] ))
		darkexptimes.append(darkexptime)
		deltime.append(np.abs(ic1.summary['mjd'][0]-darkmjd))
	if 'KCT' in obs:
		indx_closet = np.where(
			(np.abs(np.array(darkexptimes)-exptime) == np.min(np.abs(np.array(darkexptimes)-exptime)))
		)
	else:
		indx_closet = np.where(
			(deltime == np.min(deltime)) &
			(darkexptimes == np.max(darkexptimes))
		)
	if len(indx_closet[0]) == 0:
		indx_closet = np.where(
			(deltime == np.min(deltime))
		)
	else:
		pass
	# tmpdark = path_data+'/'+os.path.basename(pastdark[indx_closet].item())
	# tmpdark = '{}/{}'.format(path_data, os.path.basename(pastdark[indx_closet[0]].item()))
	# tmpdark = pastdark[indx_closet[0]].item()
	tmpdark = pastdark[indx_closet][-1]
	exptime = int(fits.getheader(tmpdark)['exptime'])

	# cpcom = 'cp {} {}/dark-{}.fits'.format(tmpdark, path_data, int(exptime))
	cpcom = 'cp {} {}'.format(tmpdark, path_data, int(exptime))
	print(cpcom)
	os.system(cpcom)
	if 'KCT' in obs:
		#KCT Exception
		mdark = CCDData.read(tmpdark, hdu=0, unit='adu')
	elif obs == 'RASA36':
		if (mode == 'high') & (badmode == True):
			mdark = CCDData.read(tmpdark, hdu=0).multiply(20)
			print('[Bad mode] Multiply 20 on the high mode dark.')
		else:
			mdark = CCDData.read(tmpdark, hdu=0)
	elif obs == 'LSGT':
		mdark = CCDData.read(tmpdark, hdu=0, unit='adu')
	else:
		mdark = CCDData.read(tmpdark, hdu=0)#, unit='adu')
	mdark.meta['FILENAME'] = os.path.basename(tmpdark)
	mdark.meta['EXPTIME'] = exptime
	darkdict['{}'.format(int(exptime))] = mdark


NO DARK FRAMES

cp /data3/paek/factory/master_frames/LSGT_ASI1600MM/dark/180-20230515-dark.fits /data4/gecko/factory/lsgt_asi1600mm/gwtest2


- Flat

In [15]:
flatdict = dict()
try:
	flatfilterlist = list(set(ic1.filter(imagetyp='flat').summary['filter']))
	for i, filte in enumerate(flatfilterlist):
		# print(i, filte)
		print('MAKING MASTER FLAT IN {}-BAND'.format(filte))
		mflat = calib.master_flat(ic1, mzero, filte, mdark=mdark, fig=True)
		flatdict[filte] = mflat

		date = fits.getheader(f'{path_data}/dark-{int(exptime)}.fits')['date-obs'][:10].replace('-', '')
		if obs == 'RASA36':
			flatim = f'{path_mframe}/{obs}/flat/{date}-n{filte}_{mode}.fits'
		else:
			flatim = f'{path_mframe}/{obs}/flat/{date}-n{filte}.fits'
		cpcom = f'cp {path_data}/n{filte}.fits {flatim}'
		print(cpcom)
		os.system(cpcom)
		plt.close('all')
except:
	print('No flat calibration image.')
	# flatdict['None'] = None
	pass
# tdict['masterframe'] = time.time() - st
protbl['status'][protbl['process']=='master_frame'] = True
protbl['time'][protbl['process']=='master_frame'] = int(time.time() - st)
#------------------------------------------------------------
#	OBJECT CALIBARTION (ZERO, DARK, FLAT)
#------------------------------------------------------------
st_ = time.time()
comment     = '='*60+'\n' \
			+ 'OBJECT CALIBRATION\n' \
			+ '='*60+'\n'
print(comment)
objfilterlist = sorted(list(set(ic1.filter(imagetyp='object').summary['filter'])))
objexptimelist = sorted(list(set(ic1.filter(imagetyp='object').summary['exptime'])))
for i, filte in enumerate(objfilterlist):
	print('PRE PROCESS FOR {} FILTER OBJECT\t[{}/{}]'.format(filte, i+1, len(objfilterlist)))
	if filte in flatdict.keys():
		mflat = flatdict[filte]
	else:
		print('\nNO {} FLAT FRAMES\n'.format(filte))
		#	CALCULATE CLOSEST ONE FROM TIME DIFFERENCE
		deltime = []
		if obs != 'RASA36':
			pastflat = np.array(glob.glob('{}/{}/flat/*n{}*.fits'.format(path_mframe, obs, filte)))
			for date in pastflat:
				flatmjd = calib.isot_to_mjd((os.path.basename(date)).split('-')[0])
				deltime.append(np.abs(ic1.summary['mjd'][0]-flatmjd))
		elif obs == 'RASA36':
			pastflat = np.array(glob.glob(f'{path_mframe}/{obs}/flat/*_{mode}-n{filte}*.fits'))
			for date in pastflat:
				flatmjd = fits.getheader(date)['MJD']
				deltime.append(np.abs(ic1.summary['mjd'][0]-flatmjd))

		indx_closet = np.where(deltime == np.min(deltime))
		tmpflat = '{}/{}'.format(path_data, os.path.basename(pastflat[indx_closet][0].item()))
		# tmpflat = pastflat[indx_closet][0].item()
		cpcom = 'cp {} {}'.format(pastflat[indx_closet][0].item(), tmpflat)
		print(cpcom)
		os.system(cpcom)
		if ('KCT' not in obs) & (obs != 'LSGT'):
			mflat = CCDData.read(tmpflat, hdu=0)#, unit='adu')
		elif obs == 'LSGT':
			mflat = CCDData.read(tmpflat, hdu=0, unit='adu')
		else:
			#KCT Exception
			mflat = CCDData.read(tmpflat, hdu=0, unit='adu')
			mflat.meta['FILENAME'] = os.path.basename(tmpflat)
		mflat.meta['FILENAME'] = os.path.basename(tmpflat)
		flatdict[filte] = mflat
	for expt in objexptimelist:
		if str(int(expt)) in darkdict.keys():
			mdark = darkdict[str(int(expt))]
		else:
			mdark = darkdict[list(darkdict.keys())[-1]]
		calib.calibration(ic1, mzero, mflat, filte, mdark=mdark)
# tdict['objectcorrection'] = time.time() - st - tdict[list(tdict.keys())[-1]]
protbl['status'][protbl['process']=='pre_process'] = True
protbl['time'][protbl['process']=='pre_process'] = int(time.time() - st_)
#	Corrected image list
fzimlist = []
for ims in ('{}/fz*.fits'.format(path_data), '{}/fz*.fit'.format(path_data), '{}/fz*.fts'.format(path_data)):
	fzimlist.extend(sorted(glob.glob(ims)))

No flat calibration image.
OBJECT CALIBRATION

PRE PROCESS FOR r FILTER OBJECT	[1/1]

NO r FLAT FRAMES

cp /data3/paek/factory/master_frames/LSGT_ASI1600MM/flat/20230515-nr.fits /data4/gecko/factory/lsgt_asi1600mm/gwtest2/20230515-nr.fits
------------------------------------------------------------
OBJECT CORRECTION
------------------------------------------------------------



### WCS Calculation

In [16]:
def convert_ra(ra_str):
    ra_hours, ra_minutes, ra_seconds = map(float, ra_str.split())
    ra_degrees = (ra_hours * 15) + (ra_minutes * 15 / 60) + (ra_seconds * 15 / 3600)
    return ra_degrees

def convert_dec(dec_str):
    dec_degrees, dec_minutes, dec_seconds = map(float, dec_str.split())
    dec_degrees = dec_degrees + (dec_minutes / 60) + (dec_seconds / 3600)
    return dec_degrees


In [17]:
#------------------------------------------------------------
#	ASTROMETRY
#------------------------------------------------------------
st_ = time.time()
print('ASTROMETRY START')
print('='*60)
astrometryfailist = []
# fzimlist = sorted(glob.glob(path_data+'/fz*.fits'))
astimlist = []
astotlist = []
astralist = []
astdelist = []

if project == 'IMSNG':
	matching_radius = obsinfo['fov']/60
	for inim in fzimlist:
		obj = (fits.getheader(inim)['object']).upper()
		# _obj = fits.getheader(inim)['object']
		# #	GW event
		# if (_obj[:2] == 'S2') | (_obj[:3] == 'MS2'):
		# 	pass
		# else:
		# 	obj = _obj.upper()
		if (obj in alltbl['obj']):
			indx_target = np.where(obj == alltbl['obj'])[0][0]
			ra, dec = alltbl['ra'][indx_target].item(), alltbl['dec'][indx_target].item()
			astimlist.append(inim)
			astralist.append(ra)
			astdelist.append(dec)
		else:
			astotlist.append(inim)

elif project == 'GECKO':
	matching_radius = 5
	for inim in fzimlist:
		hdr = fits.getheader(inim)
		if ('RA' in hdr.keys()) & ('DEC' in hdr.keys()):
			ra, dec = hdr['RA'], hdr['DEC']
			astimlist.append(inim)
			astralist.append(convert_ra(ra))
			astdelist.append(convert_dec(dec))
		else:
			astotlist.append(inim)

#	Astrometry (Known field)
if __name__ == '__main__':
	with multiprocessing.Pool(processes=ncores) as pool:
		# results = pool.starmap(calib.astrometry, zip(astimlist, repeat(obsinfo['pixscale']), astralist, astdelist, repeat(obsinfo['fov']/60.), repeat(15)))
		results = pool.starmap(calib.astrometry, zip(astimlist, repeat(obsinfo['pixscale']), astralist, astdelist, repeat(matching_radius), repeat(15)))
#	Astrometry (Unknown field)
if __name__ == '__main__':
	with multiprocessing.Pool(processes=ncores) as pool:
		results = pool.starmap(calib.astrometry, zip(astotlist, repeat(obsinfo['pixscale']), repeat(None), repeat(None), repeat(None), repeat(60)))
#	Astrometry (failed IMSNG field)
astfailist = []
for inim in astimlist:
	if (os.path.exists('{}/a{}'.format(path_data, os.path.basename(inim))) == False):
		astfailist.append(inim)
if __name__ == '__main__':
	with multiprocessing.Pool(processes=ncores) as pool:
		results = pool.starmap(calib.astrometry, zip(astfailist, repeat(obsinfo['pixscale']), repeat(None), repeat(None), repeat(None), repeat(60)))
for inim in astfailist:
	if (os.path.exists('{}/a{}'.format(path_data, os.path.basename(inim))) == False):
		astrometryfailist.append('{}/a{}'.format(path_data, os.path.basename(inim)))

os.system('rm '+path_data+'/*.axy '+path_data+'/*.corr '+path_data+'/*.xyls '+path_data+'/*.match '+path_data+'/*.rdls '+path_data+'/*.solved '+path_data+'/*.wcs ')
print('ASTROMETRY COMPLETE\n'+'='*60)
# tdict['astronomy'] = time.time() - st - tdict[list(tdict.keys())[-1]]
protbl['status'][protbl['process']=='astrometry'] = True
protbl['time'][protbl['process']=='astrometry'] = int(time.time() - st_)

ASTROMETRY START
/usr/local/astrometry/bin/solve-field /data4/gecko/factory/lsgt_asi1600mm/gwtest2/fzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit --scale-unit arcsecperpix --scale-low 0.24120000000000003 --scale-high 0.2948 --no-plots --new-fits /data4/gecko/factory/lsgt_asi1600mm/gwtest2/afzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit --overwrite --use-sextractor --cpulimit 15 --ra 307.94166666666666 --dec -83.94166666666666 --radius 5
Reading input file 1 of 1: "/data4/gecko/factory/lsgt_asi1600mm/gwtest2/fzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit"...
Extracting sources...



> WARNING: default.sex not found, using internal defaults

> 
----- SExtractor 2.19.5 started on 2023-06-01 at 21:48:22 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for fzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit
----- Measuring from: fzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit [0/3]
      "S230518h" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background 

Sorting brightness using MAG_AUTO and BACKGROUND columns failed; falling back to MAG_AUTO.
Reading sort column "MAG_AUTO"
Sorting sort column
mmapping input file
Copying table header.
Writing row 0
Done
Solving...
Reading file "/data4/gecko/factory/lsgt_asi1600mm/gwtest2/fzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.axy"...
Only searching for solutions within 5 degrees of RA,Dec (307.942,-83.9417)
Field 1 did not solve (index index-4207-11.fits, field objects 1-10).
Field 1 did not solve (index index-4207-10.fits, field objects 1-10).
Field 1 did not solve (index index-4207-08.fits, field objects 1-10).
Field 1 did not solve (index index-4206-11.fits, field objects 1-10).
Field 1 did not solve (index index-4206-10.fits, field objects 1-10).
Field 1 did not solve (index index-4206-08.fits, field objects 1-10).
Field 1 did not solve (index index-4205-11.fits, field objects 1-10).
Field 1 did not solve (index index-4205-10.fits, field objects 1-10).
Field 1 did not solve 


> WARNING: default.sex not found, using internal defaults

> 
----- SExtractor 2.19.5 started on 2023-06-01 at 21:48:42 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for fzT52-ceouobs.sophia-S230518h-20230519-001052-r-BIN1-E-180-002.fit
----- Measuring from: fzT52-ceouobs.sophia-S230518h-20230519-001052-r-BIN1-E-180-002.fit [0/3]
      "S230518h" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background 

Sorting brightness using MAG_AUTO and BACKGROUND columns failed; falling back to MAG_AUTO.
Reading sort column "MAG_AUTO"
Sorting sort column
mmapping input file
Copying table header.
Writing row 0
Done
Solving...
Reading file "/data4/gecko/factory/lsgt_asi1600mm/gwtest2/fzT52-ceouobs.sophia-S230518h-20230519-001052-r-BIN1-E-180-002.axy"...
Only searching for solutions within 5 degrees of RA,Dec (307.942,-83.9417)
Field 1 did not solve (index index-4207-11.fits, field objects 1-10).
Field 1 did not solve (index index-4207-10.fits, field objects 1-10).
Field 1 did not solve (index index-4207-08.fits, field objects 1-10).
Field 1 did not solve (index index-4206-11.fits, field objects 1-10).
Field 1 did not solve (index index-4206-10.fits, field objects 1-10).
Field 1 did not solve (index index-4206-08.fits, field objects 1-10).
Field 1 did not solve (index index-4205-11.fits, field objects 1-10).
Field 1 did not solve (index index-4205-10.fits, field objects 1-10).
Field 1 did not solve 

...

Sorting brightness using MAG_AUTO and BACKGROUND columns failed; falling back to MAG_AUTO.
Reading sort column "MAG_AUTO"
Sorting sort column
mmapping input file
Copying table header.
Writing row 0
Done
Solving...
Reading file "/data4/gecko/factory/lsgt_asi1600mm/gwtest2/fzT52-ceouobs.sophia-S230518h_1-20230519-025343-r-BIN1-E-180-001.axy"...
Only searching for solutions within 5 degrees of RA,Dec (301.117,-83.81)
Field 1 did not solve (index index-4207-11.fits, field objects 1-10).
Field 1 did not solve (index index-4207-10.fits, field objects 1-10).
Field 1 did not solve (index index-4206-11.fits, field objects 1-10).
Field 1 did not solve (index index-4206-10.fits, field objects 1-10).
  log-odds ratio 165.405 (6.83341e+71), 22 match, 0 conflict, 14 distractors, 26 index.
  RA,Dec = (301.575,-84.2345), pixel scale 0.268641 arcsec/pix.
  Hit/miss:   Hit/miss: ++++++++-+-+++++-+-+-+-+-+-+---+---+(best)----------------------------------------------------------------
Field 1: solved wit


> WARNING: default.sex not found, using internal defaults

> 
----- SExtractor 2.19.5 started on 2023-06-01 at 21:52:11 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for fzT52-ceouobs.sophia-S230518h_1-20230519-025702-r-BIN1-E-180-002.fit
----- Measuring from: fzT52-ceouobs.sophia-S230518h_1-20230519-025702-r-BIN1-E-180-002.fit [0/3]
      "S230518h" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up backgro

Sorting brightness using MAG_AUTO and BACKGROUND columns failed; falling back to MAG_AUTO.
Reading sort column "MAG_AUTO"
Sorting sort column
mmapping input file
Copying table header.
Writing row 0
Done
Solving...
Reading file "/data4/gecko/factory/lsgt_asi1600mm/gwtest2/fzT52-ceouobs.sophia-S230518h_1-20230519-025702-r-BIN1-E-180-002.axy"...
Only searching for solutions within 5 degrees of RA,Dec (301.117,-83.81)
Field 1 did not solve (index index-4207-11.fits, field objects 1-10).
Field 1 did not solve (index index-4207-10.fits, field objects 1-10).
Field 1 did not solve (index index-4206-11.fits, field objects 1-10).
Field 1 did not solve (index index-4206-10.fits, field objects 1-10).
  log-odds ratio 165.502 (7.5251e+71), 22 match, 0 conflict, 14 distractors, 26 index.
  RA,Dec = (301.576,-84.2346), pixel scale 0.268651 arcsec/pix.
  Hit/miss:   Hit/miss: ++++++++-+-+++++-+-+-+-+-+-+---+---+(best)----------------------------------------------------------------
Field 1: solved with


> WARNING: default.sex not found, using internal defaults

> 
----- SExtractor 2.19.5 started on 2023-06-01 at 21:52:19 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for fzT52-ceouobs.sophia-S230518h_1-20230519-030016-r-BIN1-E-180-003.fit
----- Measuring from: fzT52-ceouobs.sophia-S230518h_1-20230519-030016-r-BIN1-E-180-003.fit [0/3]
      "S230518h" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up backgro

Sorting brightness using MAG_AUTO and BACKGROUND columns failed; falling back to MAG_AUTO.
Reading sort column "MAG_AUTO"
Sorting sort column
mmapping input file
Copying table header.
Writing row 0
Done
Solving...
Reading file "/data4/gecko/factory/lsgt_asi1600mm/gwtest2/fzT52-ceouobs.sophia-S230518h_1-20230519-030016-r-BIN1-E-180-003.axy"...
Only searching for solutions within 5 degrees of RA,Dec (301.117,-83.81)
Field 1 did not solve (index index-4207-11.fits, field objects 1-10).
Field 1 did not solve (index index-4207-10.fits, field objects 1-10).
Field 1 did not solve (index index-4206-11.fits, field objects 1-10).
Field 1 did not solve (index index-4206-10.fits, field objects 1-10).
  log-odds ratio 165.092 (4.99691e+71), 22 match, 0 conflict, 14 distractors, 26 index.
  RA,Dec = (301.577,-84.2347), pixel scale 0.268668 arcsec/pix.
  Hit/miss:   Hit/miss: ++++++++-+-+++++-+---+-+-+-+---+-+-+(best)----------------------------------------------------------------
Field 1: solved wit

### Cosmic-ray Removal

In [18]:
st_ = time.time()
print('Quick seeing measurement with SE & Cosmic ray removal')
print('='*60)
gain = ccdinfo['gain'].value
rdnoise = ccdinfo['rdnoise']

# afzimlist = sorted(glob.glob(path_data+'/afz*.fits'))
afzimlist = []
for ims in ('{}/a*.fits'.format(path_data), '{}/a*.fit'.format(path_data), '{}/a*.fts'.format(path_data)):
	afzimlist.extend(sorted(glob.glob(ims)))
outimlist = []
for i, inim in enumerate(afzimlist):
	outim = '{}/cr{}'.format(os.path.dirname(inim), os.path.basename(inim))
	outimlist.append(outim)
if ('KCT' not in obs) & ('RASA36' not in obs) & ('LOAO_FLI' not in obs) & ('LSGT_ASI1600MM' != obs):
	#	Seeing measurement
	if __name__ == '__main__':
		with multiprocessing.Pool(processes=ncores) as pool:
			results = pool.starmap(tool.SE_seeing, zip(afzimlist, repeat(obs), repeat(path_obs), repeat(path_config), repeat(3*u.arcsecond), repeat(0.95), repeat(True)))
	#	Remove cosmic-ray
	if __name__ == '__main__':
		with multiprocessing.Pool(processes=ncores) as pool:
			results = pool.starmap(calib.cr_removal, zip(afzimlist, outimlist, repeat(gain), repeat(rdnoise)))
else:
	print('Skip Seeing measurement & CR remove processes for {}'.format(obs))
	for inim, outim in zip(afzimlist, outimlist):
		cpcom = 'cp {} {}'.format(inim, outim)
		print(cpcom)
		os.system(cpcom)
protbl['status'][protbl['process']=='cr_removal'] = True
protbl['time'][protbl['process']=='cr_removal'] = int(time.time() - st_)

Quick seeing measurement with SE & Cosmic ray removal
Skip Seeing measurement & CR remove processes for LSGT_ASI1600MM
cp /data4/gecko/factory/lsgt_asi1600mm/gwtest2/afzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit
cp /data4/gecko/factory/lsgt_asi1600mm/gwtest2/afzT52-ceouobs.sophia-S230518h-20230519-001052-r-BIN1-E-180-002.fit /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20230519-001052-r-BIN1-E-180-002.fit
cp /data4/gecko/factory/lsgt_asi1600mm/gwtest2/afzT52-ceouobs.sophia-S230518h-20230519-001406-r-BIN1-E-180-003.fit /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20230519-001406-r-BIN1-E-180-003.fit
cp /data4/gecko/factory/lsgt_asi1600mm/gwtest2/afzT52-ceouobs.sophia-S230518h-20230519-011216-r-BIN1-E-180-001.fit /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20

### Rename to IMSNG/GECKO Convention

In [19]:
fov = obsinfo['fov']*u.arcmin 
crafzimlist = []
for ims in ('{}/cra*.fits'.format(path_data), '{}/cra*.fit'.format(path_data), '{}/cra*.fts'.format(path_data)):
	crafzimlist.extend(sorted(glob.glob(ims)))
# for inim in sorted(glob.glob('{}/crafz*.fits'.format(path_data))): 
for inim in crafzimlist:
	obj = fits.getheader(inim)['object'] 
	#	Modify incorrect object header
	if (inim.replace('crafz', 'afz') in astrometryfailist) & (obj in alltbl['obj']): 
		robj, sep = tool.imsng_name_correction(inim, alltbl, radius=fov) 
	else:
		pass
	calib.fnamechange(inim, obs)

caliblist = sorted(glob.glob(path_data+'/Calib*.fits'))
ic_cal = ImageFileCollection(path_data, glob_include='Calib*0.fits', keywords='*')
os.system('chmod 777 {}'.format(path_data))
os.system('chmod 777 {}/*'.format(path_data))
#	Calib-*.fits TO SAVE PATH
f = open(path_data+'/object.txt', 'a')
f.write('obs obj dateobs filter exptime\n')
for inim in caliblist:
	img = os.path.basename(inim)
	part = img.split('-')
	line = '{} {} {} {} {}\n'.format(part[1], part[2], part[3]+'T'+part[4], part[5], part[6])
	# print(line)
	f.write(line)
f.close()

#	DATA FOLDER TO SAVE PATH
# os.system('rm {}/afz*.fits {}/fz*.fits'.format(path_data, path_data))
os.system(f'rm {path_data}/*fz*.f*')
os.system(f'rm -rf {path_save}/{os.path.basename(path_data)}')
plt.close('all')

mv /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20230519-000727-r-BIN1-E-180-001.fit /data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h-20230519-140739-r-180.fits
mv /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20230519-001052-r-BIN1-E-180-002.fit /data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h-20230519-141055-r-180.fits
mv /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20230519-001406-r-BIN1-E-180-003.fit /data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h-20230519-141409-r-180.fits
mv /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20230519-011216-r-BIN1-E-180-001.fit /data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h-20230519-151218-r-180.fits
mv /data4/gecko/factory/lsgt_asi1600mm/gwtest2/crafzT52-ceouobs.sophia-S230518h-20230519-011530-r-BIN1-E-180-002.fit /data4/geck

- RA, Dec, Polygon Info

In [20]:
if obsmode == 'TILING':
	obsprefix = obs.split('_')[0]

	for calibim in caliblist:
		center, vertices = tool.get_wcs_coordinates(calibim)
		c_center = SkyCoord(center[0].item(), center[1].item(), unit='deg')

		indx, sep, _ = c_center.match_to_catalog_sky(c_skygrid)

		# if sep.arcmin > obsinfo['fov']/10:
		# 	print(f'Wrong Pointing ({sep.arcmin[0]:.1f} arcmin)')

		nid = skygridtbl['#id'][indx]
		ratile = skygridtbl['ra'][indx]
		dectile = skygridtbl['dec'][indx]

		# print(f"{nid}: {ratile:.3f} {dectile:.3f}")

		#	GW Information
		fits.setval(calibim, "GWEVENT", value=fits.getheader(calibim)['OBJECT'], comment="GW EVENT NAME")
		#	Temporaliy change the OBJECT header
		fits.setval(calibim, "OBJECT", value=f"{obsprefix}_TILE_{nid}",)

		#	Put Tile Information
		fits.setval(calibim, "IDTILE", value=nid, comment="ID CLOSEST TILE")
		fits.setval(calibim, "SEPTILE", value=sep.arcmin[0], comment="SEP CLOSEST TILE")
		fits.setval(calibim, "RATILE", value=round(center[0].item(), 3), comment="RA CLOSEST TILE [deg]")	
		fits.setval(calibim, "DECTILE", value=round(center[1].item(), 3), comment="DEC CLOSEST TILE [deg]")

		#	Put Center & Polygon Infomation
		fits.setval(calibim, "RACENT", value=round(center[0].item(), 3), comment="RA CENTER [deg]")	
		fits.setval(calibim, "DECCENT", value=round(center[1].item(), 3), comment="DEC CENTER [deg]")	
		for ii, (_ra, _dec) in enumerate(vertices):
			# print(ii, _ra, _dec)
			fits.setval(calibim, f"RAPOLY{ii}", value=round(_ra, 3), comment=f"RA POLYGON {ii} [deg]")	
			fits.setval(calibim, f"DEPOLY{ii}", value=round(_dec, 3), comment=f"DEC POLYGON {ii} [deg]")
		
else:
	for calibim in caliblist:

		center, vertices = tool.get_wcs_coordinates(calibim)

		fits.setval(calibim, "RACENT", value=round(center[0].item(), 3), comment="RA CENTER [deg]")	
		fits.setval(calibim, "DECCENT", value=round(center[1].item(), 3), comment="DEC CENTER [deg]")	
		
		for ii, (_ra, _dec) in enumerate(vertices):
			# print(ii, _ra, _dec)
			fits.setval(calibim, f"RAPOLY{ii}", value=round(_ra, 3), comment=f"RA POLYGON {ii} [deg]")	
			fits.setval(calibim, f"DEPOLY{ii}", value=round(_dec, 3), comment=f"DEC POLYGON {ii} [deg]")

		#	Matching with host galaxy candidate catalog
		c_center = SkyCoord(center[0].item(), center[1].item(), unit='deg')
		indx, sep, _ = c_center.match_to_catalog_sky(c_hg)
		#	Should we put the condition about the pointing separation?
		# --> if sep.arcmin < fov.value:
		target = f"G{hgtbl['name'][indx]}"
		raobj = hgtbl['ra'][indx]
		decobj = hgtbl['dec'][indx]
		print(f"{os.path.basename(calibim)} {target} {raobj} {decobj} (sep={sep.arcmin[0]:.3f} arcmin)")

		#	GW Information
		fits.setval(calibim, "GWEVENT", value=fits.getheader(calibim)['OBJECT'], comment="GW EVENT NAME")
		#	Temporaliy change the OBJECT header
		fits.setval(calibim, "OBJECT", value=target,)

		fits.setval(calibim, "TARGRA", value=round(center[0].item(), 3), comment="TARGET RA [deg]")	
		fits.setval(calibim, "TARGDEC", value=round(center[1].item(), 3), comment="TARGET DEC [deg]")	
		fits.setval(calibim, "TARGOFF", value=round(sep.arcmin[0], 3), comment="OFFSET BETW TARGET AND POINTING [arcmin]")	

		newim = calibim.replace(gwobj, f"{gwobj}_{target}")
		os.system(f"mv {calibim} {newim}")

Calib-LSGT_ASI1600MM-S230518h-20230519-114025-r-180.fits G1527565 153.695 -88.462 (sep=0.044 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-114346-r-180.fits G1527565 153.695 -88.462 (sep=0.045 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-114701-r-180.fits G1527565 153.695 -88.462 (sep=0.054 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-122945-r-180.fits G1302073 293.889 -80.75 (sep=2.517 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-123300-r-180.fits G1302073 293.889 -80.75 (sep=2.536 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-123614-r-180.fits G1302073 293.889 -80.75 (sep=2.545 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-131842-r-180.fits G1279684 301.137 -83.118 (sep=0.832 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-132156-r-180.fits G1279684 301.137 -83.118 (sep=0.841 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-132511-r-180.fits G1279684 301.137 -83.118 (sep=0.853 arcmin)
Calib-LSGT_ASI1600MM-S230518h-20230519-140739-r-180.fits G18642070 308.617 -84.07 (sep=2.328 a

### Defringe
- Only for LOAO z, I, Y-bands

In [21]:
st_ = time.time()
if (obs == 'LOAO') & ('I' in ic_cal.filter(imagetyp='object').summary['filter']):
	dfim = '/home/paek/qsopy/fringe/LOAO/fringe_i_ori.fits'
	dfdat = '/home/paek/qsopy/fringe/LOAO/fringe_i.dat'
	dfimlist = []
	for inim in ic_cal.filter(imagetyp='object', filter='I').summary['file']:
		# dfimlist.append(calib.defringe(str(inim), dfim, dfdat))
		dfedim = calib.defringe(str(inim), dfim, dfdat)
		mvcom = 'mv {} {}'.format(dfedim, inim)
		print(mvcom)
		os.system(mvcom)
	# tdict['defringe'] = time.time() - st - tdict[list(tdict.keys())[-1]]
else:
	print('No images to defringe')
	pass
protbl['status'][protbl['process']=='defringe'] = True
protbl['time'][protbl['process']=='defringe'] = int(time.time() - st_)
#------------------------------------------------------------
print('='*60)
print('Calibration IS DONE.\t('+str(int(time.time() - starttime))+' sec)')
print('='*60)

No images to defringe
Calibration IS DONE.	(426 sec)


## Photometry

In [22]:
st_ = time.time()
print('#\tPhotometry')
path_infile = f'{path_data}/{os.path.basename(path_default_gphot)}'
path_new_gphot = f'{os.path.dirname(path_infile)}/gphot.config'

#	Copy default photometry configuration
cpcom = f'cp {path_default_gphot} {path_new_gphot}'
print(cpcom)
os.system(cpcom)

#	Read default photometry configuration
f = open(path_default_gphot, 'r')
lines = f.read().splitlines()
f.close()

#	Write photometry configuration
g = open(path_new_gphot, 'w')
for line in lines:
	if 'imkey' in line:
		line = f'imkey\t{path_data}/C*0.fits'
	else:
		pass
	g.write(line+'\n')
g.close()

if obs == 'DOAO':
	path_phot = path_phot_sg
else:
	path_phot = path_phot_mp

#	Execute
com = f'python {path_phot} {path_data} {ncores}'
# com = f'python {path_phot} {path_data} 1'
print(com)
os.system(com)

protbl['status'][protbl['process']=='photometry'] = True
protbl['time'][protbl['process']=='photometry'] = int(time.time() - st_)

#	Photometry
cp ./config/gphot.lsgt_asi1600mm.config /data4/gecko/factory/lsgt_asi1600mm/gwtest2/gphot.config
python ./phot/gregoryphot_mp_2021.py /data4/gecko/factory/lsgt_asi1600mm/gwtest2 1
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/gphot.config
No transient catalog for photometry
#	27 images to do photometry
0	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151218-r-180.fits
1	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151533-r-180.fits
2	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151847-r-180.fits
3	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1208712-20230519-160458-r-180.fits
4	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1208712-20230519-160813-r-180.fits
5	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1208712-20230519-161128-r-180.fits
6	/data4/geck

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
> 
----- SExtractor 2.19.5 started on 2023-06-01 at 21:52:45 with 1 thread

> Setting catalog parameters

> WARNING: Not enough apertures provided in config.:
         some APER photometric values will remain blank 

> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights

> WARNING: Not enough apertures provided in config.:
         some APER photometric values will remain blank 

> Initializing catalog
> Looking for Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151218-r-180.fits
----- Measuring from: Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151218-r-180.fits
      "G1167888" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at 

2. SOURCE EXTRACTOR
3. MATCHING
4. ZERO POINT CALCULATION for MAG_AUTO [1/7]
--------------------------------------------------
ALL					: 49
--------------------------------------------------
FLAG(<=3)				: 49
r REF. MAGCUT (12.0-20.0)		: 45
rerr REF. MAGERR CUT < 0.05		: 30
MAGERR_AUTO CUT < 0.05		: 49
TOTAL #					: 25


MAG TYP     : MAG_AUTO
ZP          = 29.348
ZP ERR      = 0.023
STD.NUMB    = 20
REJ.NUMB    = 5
CLIP UPPER  = 0
CTIP LOWER  = 5
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER [2/7]
--------------------------------------------------
ALL					: 49
--------------------------------------------------
FLAG(<=3)				: 49
r REF. MAGCUT (12.0-20.0)		: 45
rerr REF. MAGERR CUT < 0.05		: 30
MAGERR_APER CUT < 0.05		: 49
TOTAL #					: 25


MAG TYP     : MAG_APER
ZP          = 29.006
ZP ERR      = 0.032
STD.NUMB    = 23
REJ.NUMB    = 2
CLIP UPPER  = 0
CTIP LOWER  = 2
4-1. PLOT


...

MAG TYP     : MAG_APER_1
ZP          = 28.76
ZP ERR      = 0.096
STD.NUMB    = 8
REJ.NUMB    = 1
CLIP UPPER  = 0
CTIP LOWER  = 1
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER_2 [4/7]
--------------------------------------------------
ALL					: 62
--------------------------------------------------
FLAG(<=3)				: 62
r REF. MAGCUT (12.0-20.0)		: 62
rerr REF. MAGERR CUT < 0.05		: 11
MAGERR_APER_2 CUT < 0.05		: 62
TOTAL #					: 9


MAG TYP     : MAG_APER_2
ZP          = 29.042
ZP ERR      = 0.044
STD.NUMB    = 6
REJ.NUMB    = 3
CLIP UPPER  = 0
CTIP LOWER  = 3
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER_3 [5/7]
--------------------------------------------------
ALL					: 62
--------------------------------------------------
FLAG(<=3)				: 62
r REF. MAGCUT (12.0-20.0)		: 62
rerr REF. MAGERR CUT < 0.05		: 11
MAGERR_APER_3 CUT < 0.05		: 62
TOTAL #					: 9


MAG TYP     : MAG_APER_3
ZP          = 29.016
ZP ERR      = 0.093
STD.NUMB    = 9
REJ.NUMB    = 0
CLIP UPPER  = 0
CTIP LOWER  = 0
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER_4 [6/7]
--------------------------------------------------
ALL					: 62
--------------------------------------------------
FLAG(<=3)				: 62
r REF. MAGCUT (12.0-20.0)		: 62
rerr REF. MAGERR CUT < 0.05		: 11
MAGERR_APER_4 CUT < 0.05		: 62
TOTAL #					: 9


MAG TYP     : MAG_APER_4
ZP          = 27.56
ZP ERR      = 0.091
STD.NUMB    = 8
REJ.NUMB    = 1
CLIP UPPER  = 0
CTIP LOWER  = 1
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER_5 [7/7]
--------------------------------------------------
ALL					: 62
--------------------------------------------------
FLAG(<=3)				: 62
r REF. MAGCUT (12.0-20.0)		: 62
rerr REF. MAGERR CUT < 0.05		: 11
MAGERR_APER_5 CUT < 0.05		: 62
TOTAL #					: 9


MAG TYP     : MAG_APER_5
ZP          = 28.389
ZP ERR      = 0.099
STD.NUMB    = 8
REJ.NUMB    = 1
CLIP UPPER  = 0
CTIP LOWER  = 1
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
5. RESULT TO FILES
0 failed images
PHOTOMETRY IS DONE.	(3.683 mins)


## Image registering & combine

In [23]:
st_ = time.time()
print('IMAGE REGISTERING & COMBINE')
combined_images = []
step = (1/24/60)*60	# 1 hour
ic_cal_phot = ImageFileCollection(path_data, glob_include='Calib*0.fits', keywords='*')
calist = sorted(glob.glob('{}/Calib*.fits'.format(path_data)))
objlist = sorted(list(set(ic_cal_phot.summary['object'])))
filterlist = sorted(list(set(ic_cal_phot.summary['filter'])))
# obj = 'NGC3147'
# filte = 'R'

IMAGE REGISTERING & COMBINE


In [24]:

for obj in objlist:
	for filte in filterlist:
		# imlist_tmp = sorted(glob.glob('{}/Calib*-{}-*-{}-*.fits'.format(path_data, obj, filte)))
		imlist_tmp = list(ic_cal_phot.filter(object=obj, filter=filte).summary['file'].copy())
		if len(imlist_tmp) == 0:
			pass
		elif len(imlist_tmp) == 1:
			inim = imlist_tmp[0]
			comim = inim.replace('.fits', '.com.fits')
			cpcom = f'cp {inim} {comim}'
			print(cpcom)
			os.system(cpcom)
		else:
			print(obj, filte)
			# ic_part = sorted(glob.glob('{}/Calib*{}*{}*.fits'.format(path_data, obj, filte)))

			jds = np.array([fits.getheader(inim)['jd'] for inim in imlist_tmp])
			delts = jds - np.min(jds)

			grouplist = []
			grouplists = []

			i = 0
			for i in range(len(delts)):
				#	Initial setting
				if i == 0:
					t0 = delts[i]
				#	Add last group to grouplists
				elif i == len(delts)-1:
					grouplists.append(grouplist)
				t1 = delts[i]
				# print(t0, t1)
				dif = np.abs(t0-t1)

				if dif < step:
					grouplist.append(imlist_tmp[i])
				#	Generate new group
				else:
					grouplists.append(grouplist)
					grouplist = [imlist_tmp[i]]	 
				t0 = t1
				
			for group in grouplists:
				print('-'*60)
				if len(group) > 1:
					ref_image = group[0]
					images_to_align = group[1:]
					for inim in group:
						print(inim)
					# _data, _hdr = fits.getdata(ref_image, header=True)
					# ximage, yimage = _data.shape
					# racent, decent = _hdr['RACENT'], _hdr['DECCENT']
					try:
						# outim = tool.swarpcomb(
						# 	images_to_align,
						# 	gain=obsinfo['gain'].value,
						# 	pixscale=obsinfo['pixscale'],
						# 	racent=racent,
						# 	decent=decent,
						# 	ximage=ximage,
						# 	yimage=yimage,
						# 	listname='obj.list',
						# 	path_save='.',
						# 	keys_to_get=[
						# 		'OBJECT',
						# 		'FILTER',
						# 		'RACENT',
						# 		'DECCENT',
						# 		'RAPOLY0',
						# 		'DEPOLY0',
						# 		'RAPOLY1',
						# 		'DEPOLY1',
						# 		'RAPOLY2',
						# 		'DEPOLY2',
						# 		'RAPOLY3',
						# 		'DEPOLY3',
						# 		]
						# 	)
						outim = tool.imcombine_routine(images_to_align, ref_image)
						combined_images.append(outim)
					except:
						print('Fail to image align & combine routine.')
						print(images_to_align)
						pass
				else:
					print('There is only one image.')
					combined_images.append(group[0])

protbl['status'][protbl['process']=='image_stack'] = True
protbl['time'][protbl['process']=='image_stack'] = int(time.time() - st_)

G1167888 r
------------------------------------------------------------
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151218-r-180.fits
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151533-r-180.fits
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151847-r-180.fits


Reference image	: /data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151218-r-180.fits
Input images	:
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151533-r-180.fits
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151847-r-180.fits
#	IMAGE REGISTERING
##########  Preparing reference ...
No filter file found, using default filter
No NNW file found, using default NNW config


> 
----- SExtractor 2.19.5 started on 2023-06-01 at 21:56:29 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for .pysex.fits
----- Measuring from: .pysex.fits
      "Unnamed" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768
> Setting up background map at line:  832
> Setting up background map at line:  896
> Setting up background map at line:  960
> Setting up background map at li

Number of sources in catalog : 253
EXT_NUMBER values found in catalog : 1
I've selected 242 sources
Making more quads, from quadlevel 0 ...
Made   35 quads from  242 stars (combi n=7 s=0 d=30.0)
Removing 0/35 duplicates
########## Processing /data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151533-r-180.fits
No filter file found, using default filter
No NNW file found, using default NNW config


> 
----- SExtractor 2.19.5 started on 2023-06-01 at 21:56:30 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for .pysex.fits
----- Measuring from: .pysex.fits
      "Unnamed" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768
> Setting up background map at line:  832
> Setting up background map at line:  896
> Setting up background map at line:  960
> Setting up background map at li

...

Number of sources in catalog : 253
EXT_NUMBER values found in catalog : 1
I've selected 240 sources
Making more quads, from quadlevel 0 ...
Made   35 quads from  240 stars (combi n=7 s=0 d=30.0)
Removing 0/35 duplicates
Finding 4 best candidates among 35 x 35 (ukn x ref)
We have a maximum of 35 quad pairs
Cand  1 (dist.   0.00001688) : Rotation   +0.018967 [deg], scale 1.000018
Cand  2 (dist.   0.00001814) : Rotation   +0.018967 [deg], scale 1.000018
Cand  3 (dist.   0.00001855) : Rotation   +0.018899 [deg], scale 1.000030
Cand  4 (dist.   0.00002457) : Rotation   +0.018967 [deg], scale 1.000018
227/240 stars with distance < r = 5.0 (mean 0.7, median 0.3, std 0.9)
Filtered for companions, keeping 226/227 matches
Refitting transform (before/after) :
Rotation   +0.018967 [deg], scale 1.000018
Rotation   +0.017508 [deg], scale 0.999980
227/240 stars with distance < r = 5.0 (mean 0.6, median 0.3, std 0.8)
Filtered for companions, keeping 226/227 matches
I'm done !
Computed flux ratio from 

> 
----- SExtractor 2.19.5 started on 2023-06-01 at 22:00:27 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for .pysex.fits
----- Measuring from: .pysex.fits
      "Unnamed" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768
> Setting up background map at line:  832
> Setting up background map at line:  896
> Setting up background map at line:  960
> Setting up background map at li

Number of sources in catalog : 183
EXT_NUMBER values found in catalog : 1
I've selected 178 sources
Making more quads, from quadlevel 0 ...
Made   35 quads from  178 stars (combi n=7 s=0 d=30.0)
Removing 0/35 duplicates
########## Processing /data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G18642070-20230519-184954-r-180.fits
No filter file found, using default filter
No NNW file found, using default NNW config


> 
----- SExtractor 2.19.5 started on 2023-06-01 at 22:00:28 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for .pysex.fits
----- Measuring from: .pysex.fits
      "Unnamed" / no ext. header / 4656x3520 / 64 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768
> Setting up background map at line:  832
> Setting up background map at line:  896
> Setting up background map at line:  960
> Setting up background map at li

Number of sources in catalog : 175
EXT_NUMBER values found in catalog : 1
I've selected 170 sources
Making more quads, from quadlevel 0 ...
Made   35 quads from  170 stars (combi n=7 s=0 d=30.0)
Removing 0/35 duplicates
Finding 4 best candidates among 35 x 35 (ukn x ref)
We have a maximum of 35 quad pairs
Cand  1 (dist.   0.00004143) : Rotation   +0.005141 [deg], scale 0.999915
Cand  2 (dist.   0.00005404) : Rotation   +0.003065 [deg], scale 0.999938
Cand  3 (dist.   0.00006137) : Rotation   +0.003065 [deg], scale 0.999938
Cand  4 (dist.   0.00007937) : Rotation   +0.003065 [deg], scale 0.999938
164/170 stars with distance < r = 5.0 (mean 0.9, median 0.5, std 0.9)
Filtered for companions, keeping 163/164 matches
Refitting transform (before/after) :
Rotation   +0.005141 [deg], scale 0.999915
Rotation   +0.008056 [deg], scale 0.999879
165/170 stars with distance < r = 5.0 (mean 0.8, median 0.4, std 0.9)
Filtered for companions, keeping 164/165 matches
I'm done !
Computed flux ratio from 

## Photometry for combined images

In [25]:
st_ = time.time()
#	Write photometry configuration
h = open(path_new_gphot, 'w')
for line in lines:
	if 'imkey' in line:
		line = '{}\t{}/C*com.fits'.format('imkey', path_data)
	else:
		pass
	h.write(line+'\n')
h.close()
#	Execute
path_phot = path_phot_mp
com = 'python {} {} {}'.format(path_phot, path_data, ncores)
print(com)
os.system(com)
# tdict['photometry_com'] = time.time() - st - tdict[list(tdict.keys())[-1]]
protbl['status'][protbl['process']=='photometry_com'] = True
protbl['time'][protbl['process']=='photometry_com'] = int(time.time() - st_)

ic_com_phot = ImageFileCollection(path_data, glob_include='Calib*com.fits', keywords='*')	
#	Summary
print('Draw observation summary plots')
# for filte in list(set(ic_cal_phot.summary['filter'])):
for filte in filterlist:
	try:
		tool.obs_summary(filte, ic_cal_phot, ic_com_phot, path_save=path_data)
	except:
		print('Fail to make summary plots.')
		pass
	plt.close('all')

python ./phot/gregoryphot_mp_2021.py /data4/gecko/factory/lsgt_asi1600mm/gwtest2 1
/data4/gecko/factory/lsgt_asi1600mm/gwtest2/gphot.config
No transient catalog for photometry
#	9 images to do photometry
0	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151550-r-540.com.fits
1	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1208712-20230519-160809-r-540.com.fits
2	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1269791-20230519-172038-r-1080.com.fits
3	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1279684-20230519-132204-r-540.com.fits
4	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1302073-20230519-123307-r-540.com.fits
5	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1335592-20230519-185449-r-180.com.fits
6	/data4/gecko/factory/lsgt_asi1600mm/gwtest2/Calib-LSGT_ASI1600MM-S230518h_G1527565-20230519-11434

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
> 
----- SExtractor 2.19.5 started on 2023-06-01 at 22:00:45 with 1 thread

> Setting catalog parameters

> WARNING: Not enough apertures provided in config.:
         some APER photometric values will remain blank 

> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights

> WARNING: Not enough apertures provided in config.:
         some APER photometric values will remain blank 

> Initializing catalog
> Looking for Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151550-r-540.com.fits
----- Measuring from: Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151550-r-540.com.fits
      "G1167888" / no ext. header / 4656x3520 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background

2. SOURCE EXTRACTOR
3. MATCHING
4. ZERO POINT CALCULATION for MAG_AUTO [1/7]
--------------------------------------------------
ALL					: 49
--------------------------------------------------
FLAG(<=3)				: 49
r REF. MAGCUT (12.0-20.0)		: 45
rerr REF. MAGERR CUT < 0.05		: 30
MAGERR_AUTO CUT < 0.05		: 49
TOTAL #					: 25


MAG TYP     : MAG_AUTO
ZP          = 29.331
ZP ERR      = 0.027
STD.NUMB    = 21
REJ.NUMB    = 4
CLIP UPPER  = 1
CTIP LOWER  = 3
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER [2/7]
--------------------------------------------------
ALL					: 49
--------------------------------------------------
FLAG(<=3)				: 49
r REF. MAGCUT (12.0-20.0)		: 45
rerr REF. MAGERR CUT < 0.05		: 30
MAGERR_APER CUT < 0.05		: 49
TOTAL #					: 25


MAG TYP     : MAG_APER
ZP          = 29.113
ZP ERR      = 0.021
STD.NUMB    = 20
REJ.NUMB    = 5
CLIP UPPER  = 0
CTIP LOWER  = 5
4-1. PLOT


...

MAG TYP     : MAG_APER_1
ZP          = 28.751
ZP ERR      = 0.075
STD.NUMB    = 6
REJ.NUMB    = 5
CLIP UPPER  = 0
CTIP LOWER  = 5
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER_2 [4/7]
--------------------------------------------------
ALL					: 68
--------------------------------------------------
FLAG(<=3)				: 68
r REF. MAGCUT (12.0-20.0)		: 68
rerr REF. MAGERR CUT < 0.05		: 18
MAGERR_APER_2 CUT < 0.05		: 63
TOTAL #					: 11


MAG TYP     : MAG_APER_2
ZP          = 29.038
ZP ERR      = 0.099
STD.NUMB    = 7
REJ.NUMB    = 4
CLIP UPPER  = 0
CTIP LOWER  = 4
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER_3 [5/7]
--------------------------------------------------
ALL					: 68
--------------------------------------------------
FLAG(<=3)				: 68
r REF. MAGCUT (12.0-20.0)		: 68
rerr REF. MAGERR CUT < 0.05		: 18
MAGERR_APER_3 CUT < 0.05		: 64
TOTAL #					: 12


MAG TYP     : MAG_APER_3
ZP          = 29.169
ZP ERR      = 0.097
STD.NUMB    = 7
REJ.NUMB    = 5
CLIP UPPER  = 0
CTIP LOWER  = 5
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER_4 [6/7]
--------------------------------------------------
ALL					: 68
--------------------------------------------------
FLAG(<=3)				: 68
r REF. MAGCUT (12.0-20.0)		: 68
rerr REF. MAGERR CUT < 0.05		: 18
MAGERR_APER_4 CUT < 0.05		: 65
TOTAL #					: 13


MAG TYP     : MAG_APER_4
ZP          = 27.617
ZP ERR      = 0.1
STD.NUMB    = 8
REJ.NUMB    = 5
CLIP UPPER  = 0
CTIP LOWER  = 5
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
4. ZERO POINT CALCULATION for MAG_APER_5 [7/7]
--------------------------------------------------
ALL					: 68
--------------------------------------------------
FLAG(<=3)				: 68
r REF. MAGCUT (12.0-20.0)		: 68
rerr REF. MAGERR CUT < 0.05		: 18
MAGERR_APER_5 CUT < 0.05		: 64
TOTAL #					: 12


MAG TYP     : MAG_APER_5
ZP          = 28.481
ZP ERR      = 0.069
STD.NUMB    = 6
REJ.NUMB    = 6
CLIP UPPER  = 0
CTIP LOWER  = 6
4-2. CHANGE HEADER
4-3. PHOTOMETRY TABLE
5. RESULT TO FILES
0 failed images
PHOTOMETRY IS DONE.	(1.208 mins)
Draw observation summary plots
Fail to make summary plots.


#	Image subtraction


In [26]:
ps1filte = 'r'
pixelsize = 2400

In [27]:
# if project == 'GECKO':
path_gecko = '/data7/GECKO'

# subtypelist = []

inim = combined_images[0]
hdr = fits.getheader(inim)
obj = hdr['object']
n_glade = int(obj[1:])
filte = hdr['filter']

if filte in ['R', 'r']:
	search_refimlist = glob.glob(f"{path_gecko}/*/{obs}/{filte}/Calib*{obj}*com.fits")

	#	Subtraction Type
	#	1. self: Self-consistent
	#	2. ps1: PS1
	#	3. ks4: KS4 --> TBD
	#	4. none: No Reference image
	subtype = ''
	if len(search_refimlist) > 0:
		subtype = 'self'
	else:
		#	Get it from external sources
		if len(ps1tbl[ps1tbl['GLADE']==n_glade]) > 0:
			subtype = 'ps1'
			#------------------------------------------------------------
			#	Subtraction
			#------------------------------------------------------------
			#	PS1 reference image
			tra, tdec = ps1tbl['RA'][indx_ps1][nn], ps1tbl['Dec'][indx_ps1][nn]
			#	Trim the Sci image
			hdu = snapshot_lite(inim, tra, tdec, cutsize=10)
			data = hdu.data
			data[np.isnan(data)] = hdu.header['skyval']
			#	Save
			tinim = inim.replace('fits', 'trim.fits')
			if not os.path.exists(tinim):
				hdu.writeto(tinim, overwrite=True)
			#	Ref
			refim = f'{path_ps1}/PS1_G{n_glade}_{ps1filte}_stk_{pixelsize}.fits'
			#	Subtraction
			if os.path.exists(refim) & os.path.exists(subim):
				#	WCSremap
				arefim = f"{path_save}/{os.path.basename(refim).replace('fits', 'wcsremap.fits')}"
				wcsremapcom = wcsremap(
					inim=tinim,
					refim=refim,
					outim=subim,
				)
				print(wcsremapcom)
				os.system(wcsremapcom)
				#	HOTPANTS
				hdim=f"{path_save}/hd{os.path.basename(tinim)}"
				hcim=f"{path_save}/hc{os.path.basename(tinim)}"

				hotpantscom = hotpants(
					inim=tinim,
					refim=arefim,
					outim=hdim,
					convim=hcim
					)
				print(hotpantscom)
				os.system(hotpantscom)

				ds9com = f"ds9 {subim} {hcim} {hdim} &"
				print(ds9com)
				ds9comlist.append(ds9com)
		else:
			subtype = 'none'
		print(f"{len(search_refimlist)} found for reference image --> subtype:{subtype}")


0 found for reference image --> subtype:none


In [28]:
print('IMAGE SUBTRACTION')
subtracted_images = []
ds9comlist = []
for inim in combined_images:
	hdr = fits.getheader(inim)
	# obs = os.path.basename(inim).split('-')[1]
	# obs = 'LOAO'
	obj = hdr['object']
	filte = hdr['filter']
	path_refim = '/data3/paek/factory/ref_frames/{}'.format(obs)
	refimlist = glob.glob('{}/Ref*{}*{}*.fits'.format(path_refim, obj, filte))
	if len(refimlist) > 0:
		refim = refimlist[0]

		# subim, ds9com = tool.subtraction_routine3(inim, refim)

		# if False:
		if obs not in ['LSGT', 'DOAO', 'RASA36', 'SAO_C361K',]:
			subim, ds9com = tool.subtraction_routine(inim, refim)
		else:
			subim, ds9com = tool.subtraction_routine2(inim, refim)
			if os.path.getsize(subim) != 0:
				rmcom = f"rm {subim}"
				print(rmcom)
				os.system(rmcom)
				subim, ds9com = tool.subtraction_routine(inim, refim)
			else:
				pass
		if subim != None:
			subtracted_images.append(subim)
			ds9comlist.append(ds9com)
	else:
		print('There is no reference image for {}'.format(os.path.basename(inim)))
		pass
rmcom = 'rm {}/*Ref*gregister.fits'.format(path_data)
print(rmcom)
os.system(rmcom)
# tdict['subtraction'] = time.time() - st - tdict[list(tdict.keys())[-1]]
protbl['status'][protbl['process']=='subtraction'] = True
protbl['time'][protbl['process']=='subtraction'] = int(time.time() - st_)

IMAGE SUBTRACTION
There is no reference image for Calib-LSGT_ASI1600MM-S230518h_G1167888-20230519-151550-r-540.com.fits
There is no reference image for Calib-LSGT_ASI1600MM-S230518h_G1208712-20230519-160809-r-540.com.fits
There is no reference image for Calib-LSGT_ASI1600MM-S230518h_G1269791-20230519-172038-r-1080.com.fits
There is no reference image for Calib-LSGT_ASI1600MM-S230518h_G1279684-20230519-132204-r-540.com.fits
There is no reference image for Calib-LSGT_ASI1600MM-S230518h_G1302073-20230519-123307-r-540.com.fits
There is no reference image for Calib-LSGT_ASI1600MM-S230518h_G1527565-20230519-114340-r-540.com.fits
There is no reference image for Calib-LSGT_ASI1600MM-S230518h_G18642070-20230519-141102-r-540.com.fits
There is no reference image for Calib-LSGT_ASI1600MM-S230518h_G18642070-20230519-184814-r-360.com.fits
rm /data4/gecko/factory/lsgt_asi1600mm/gwtest2/*Ref*gregister.fits


rm: cannot remove ‘/data4/gecko/factory/lsgt_asi1600mm/gwtest2/*Ref*gregister.fits’: No such file or directory


In [34]:
for inim in combined_images:
	hdr = fits.getheader(inim)
	print(hdr['OBJECT'])

G1167888
G1208712
G1269791
G1279684
G1302073
G1527565
G18642070
G18642070


##	Photometry for subtracted images


In [29]:
st_ = time.time()
#	Write photometry configuration
s = open(path_new_gphot, 'w')
for line in lines:
	if 'imkey' in line:
		# line = '{}\t{}/hd*com.fits'.format('imkey', path_data)
		line = '{}\t{}/hd*.fits'.format('imkey', path_data)
	else:
		pass
	if 'photfraction' in line:
		line = '{}\t{}'.format('photfraction', 1.0)
	else:
		pass
	if 'DETECT_MINAREA' in line:
		line = '{}\t{}'.format('DETECT_MINAREA', 10)
	else:
		pass
	if 'DETECT_THRESH' in line:
		line = '{}\t{}'.format('DETECT_THRESH', 1.25)
	else:
		pass
	s.write(line+'\n')
s.close()
#	Execute
hdimlist = sorted(glob.glob('{}/hd*.fits'.format(path_data)))
if len(hdimlist) > 0:
	com = 'python {} {}'.format(path_phot_sub, path_data)
	print(com)
	os.system(com)
	# tdict['photometry_sub'] = time.time() - st - tdict[list(tdict.keys())[-1]]
else:
	print('No subtracted image.')
	pass
protbl['status'][protbl['process']=='photometry_sub'] = True
protbl['time'][protbl['process']=='photometry_sub'] = int(time.time() - st_)

No subtracted image.


##	Transient Search


In [30]:
st_ = time.time()
fovval = fov.value
#	Input table for transient search
tstbl = Table()
# hdimlist = sorted(glob.glob(f'{path_data}/hd*com.fits'))
hdimlist = sorted(glob.glob(f'{path_data}/hd*.fits'))
if len(hdimlist) != 0:
	tstbl['hdim'] = hdimlist
	tskeys = ['hdcat', 'hcim', 'inim', 'scicat', 'refim']
	for key in tskeys:
		tstbl[key] = ' '*300
	tstbl['fovval'] = fovval

	for i, hdim in enumerate(hdimlist):
		hdcat = hdim.replace('.fits','.phot_sub.cat')
		hcim = hdim.replace('hdCalib', 'hcCalib')
		inim = hdim.replace('hdCalib', 'Calib')
		scicat = inim.replace('.fits', '.phot.cat')

		hdr = fits.getheader(hdim)
		obj = hdr['object']
		filte = hdr['filter']
		path_refim = f'/data3/paek/factory/ref_frames/{obs}'
		refimlist = glob.glob(f'{path_refim}/Ref*{obj}*{filte}*.fits')
		refim = refimlist[0]


		for key, im in zip(tskeys, [hdcat, hcim, inim, scicat, refim]):
			tstbl[key][i] = im

	out_tstbl = f'{path_data}/transient_search.txt'
	tstbl.write(out_tstbl, format='ascii.tab', overwrite=True)

	com = f'python {path_find} {out_tstbl} {ncores}'
	print(com)
	subprocess.call(com, shell=True)		


protbl['status'][protbl['process']=='transient_search'] = True
protbl['time'][protbl['process']=='transient_search'] = int(time.time() - st_)

#	Summary file

In [31]:
#------------------------------------------------------------
#------------------------------------------------------------
protbl['status'][protbl['process']=='total'] = True
protbl['time'][protbl['process']=='total'] = int(time.time() - st)	
protbl.write('{}/obs.summary.log'.format(path_data), format='ascii.tab', overwrite=True)
print(protbl)
#	Write data summary
f = open(path_data+'/obs.summary.log', 'a')
end_localtime = time.strftime('%Y-%m-%d %H:%M:%S (%Z)', time.localtime())
f.write('Pipelne start\t: {}\n'.format(start_localtime))
f.write('Pipelne end\t: {}\n'.format(end_localtime))
try:
	f.write('='*60+'\n')
	f.write('PATH :{}\n'.format(path))
	f.write('OBJECT NUMBER # :{}\n'.format(len(ic_cal.summary)))
	objkind = sorted(set(ic_cal.summary['object']))
	f.write('OBJECTS # : {}\n'.format(objkind))
	for obj in objkind:
		f.write('-'*60+'\n')
		for filte in list(set(ic_cal.summary['filter'])):
			indx_tmp = ic_cal.files_filtered(filter=filte, object=obj)
			if len(indx_tmp) > 0:
				f.write('{}\t{}\n'.format(obj, filte))
except:
	pass
f.close()

    process      status  time
                          s  
---------------- ------ -----
    master_frame   True   2.0
     pre_process   True 155.0
      astrometry   True 247.0
      cr_removal   True   4.0
        defringe   True   0.0
      photometry   True 224.0
     image_stack   True 255.0
  photometry_com   True  75.0
     subtraction   True  79.0
  photometry_sub   True   0.0
transient_search   True   0.0
           total   True 980.0


In [32]:
stop

NameError: name 'stop' is not defined

## File Transfer

In [ ]:
rmcom = 'rm {}/inv*.*'.format(path_data, path_data)
print(rmcom)
os.system(rmcom)
tails = ['.transients.', '.new.', '.ref.', '.sub.', '']
for obj in objlist:
	for filte in filterlist:
		for tail in tails:
			# tail = 'transients'
			# obj = 'NGC3147'
			# filte = 'B'

			pathto = f'{path_gal}/{obj}/{obs}/{filte}'
			files = f'{path_data}/*Calib*-{obj}-*-{filte}-*{tail}*'
			nfiles = len(glob.glob(files))
			# print(files, nfiles)
			# if nfiles >0:
			# 	print(obj, filte, pathto, files, glob.glob(files)[-1])
			if nfiles !=0:
				#	Save path
				if tail == '':
					pathto = f'{path_gal}/{obj}/{obs}/{filte}'
				else:
					pathto = f'{path_gal}/{obj}/{obs}/{filte}/transients'
				#	Make path
				if (not os.path.exists(pathto)):
					os.makedirs(pathto)
				mvcom = f'mv {files} {pathto}'
				print(mvcom)
				os.system(mvcom)
#	Image transfer
mvcom = f'mv {path_data} {path_save}'
os.system(mvcom)
#	WRITE LOG
f = open(path_log, 'a')
# f.write(path_raw+'/'+os.path.basename(path_data)+'\n')
# f.write('{}/{}\n'.format(path_raw, os.path.basename(path_data)))
f.write(f'{path_raw}/{os.path.basename(path_data)}\n')
f.close()

: 

##	Slack message

In [ ]:
total_time = round(protbl['time'][protbl['process']=='total'].item()/60., 1)

channel = '#pipeline'
text = f'[`gpPy`/{project}-{obsmode}] Processing Complete {obs} {os.path.basename(path)} Data ({nobj} objects) with {ncores} cores taking {total_time} mins'

param_slack = dict(
	token = OAuth_Token,
	channel = channel,
	text = text,
)

tool.slack_bot(**param_slack)

: 